In [34]:
import pathlib
import gpx_parser as parser
import gmplot
import mplleaflet
import matplotlib.pyplot as plt
from datetime import datetime as dt
import math

# Initializing the required dictionaries

In [36]:
rest_points = {}
rest_points2 = {}
hours = [i for i in range(0,24)]
rest_points_time = {}
rest_patterns = {}
initial_rest = {}

In [37]:
average_speed = {}

# Reading file for determining rest durations

In [38]:
def file_reading2(num):

    directory = "./"+"Sport"+"/"+"Rider"+str(num)

    count = 0
    total = 0

    for path in pathlib.Path(directory).iterdir():

        count+=1
        with open(path, 'r') as gpx_file:
            gpx = parser.parse(gpx_file)

            latitudes = []
            longitudes = []
            dates = []

            for track in gpx:
                for segment in track:
                    for point in segment:
                        total+=1
                        latitudes.append(point.latitude)
                        longitudes.append(point.longitude)
                        dates.append(point.time)


                        if(len(latitudes)>1):

                            l1 = str(dates[-2]).split(" ")
                            l2 = str(dates[-1]).split(" ")

                            d1 = l1[0]
                            d2 = l2[0]
                            t1 = l1[1]
                            t2 = l2[1]

    #                         temp = d1.split("-")
    #                         print(d1)
    #                         print(temp)

    #                         print(t1.split("-"))

                            date1 = dt(int(d1.split("-")[0]),int(d1.split("-")[1]),int(d1.split("-")[2]),
                                       int(t1.split(":")[0]),int(t1.split(":")[1]),int(t1.split(":")[2]))

                            date2 = dt(int(d2.split("-")[0]),int(d2.split("-")[1]),int(d2.split("-")[2]),
                                       int(t2.split(":")[0]),int(t2.split(":")[1]),int(t2.split(":")[2]))


                            diff = date2 - date1

                            if(diff.seconds>=1800):
                                if("1800" in rest_patterns):
                                    rest_patterns["1800"]+=1
                                else:
                                    rest_patterns["1800"]=1

                            elif(diff.seconds>=900):
                                if("900" in rest_patterns):
                                    rest_patterns["900"]+=1
                                else:
                                    rest_patterns["900"]=1

                            elif(diff.seconds>=600):
                                if("600" in rest_patterns):
                                    rest_patterns["600"]+=1
                                else:
                                    rest_patterns["600"]=1

                            elif(diff.seconds>=300):
                                if("300" in rest_patterns):
                                    rest_patterns["300"]+=1
                                else:
                                    rest_patterns["300"]=1






        if(count==400):
            break

# Reading file for all other analysis 

In [39]:
def file_reading(num):
    days=0
    directory = "./"+"Sport"+"/"+"Rider"+str(num)

    count = 0
    total = 0

    for hour in hours:
        rest_points_time[hour] = {}
        initial_rest[hour] = 0

    for path in pathlib.Path(directory).iterdir():
        count+=1
        check=False
        check_initial = False
        with open(path, 'r') as gpx_file:
            gpx = parser.parse(gpx_file)

            latitudes = []
            longitudes = []
            dates = []

            for track in gpx:
                for segment in track:
                    for point in segment:
                        total+=1
                        latitudes.append(point.latitude)
                        longitudes.append(point.longitude)
                        dates.append(point.time)
                        
                        start_hr = int(str(dates[0]).split(" ")[1].split(":")[0])

                        if(len(latitudes)>1):

                            l1 = str(dates[-2]).split(" ")
                            l2 = str(dates[-1]).split(" ")

                            d1 = l1[0]
                            d2 = l2[0]
                            t1 = l1[1]
                            t2 = l2[1]

                            date1 = dt(int(d1.split("-")[0]),int(d1.split("-")[1]),int(d1.split("-")[2]),
                                       int(t1.split(":")[0]),int(t1.split(":")[1]),int(t1.split(":")[2]))

                            date2 = dt(int(d2.split("-")[0]),int(d2.split("-")[1]),int(d2.split("-")[2]),
                                       int(t2.split(":")[0]),int(t2.split(":")[1]),int(t2.split(":")[2]))

                            diff = date2 - date1

                            hr = int(t1.split(":")[0])
                            
                            if(diff.seconds>=300):
                                loc = (latitudes[-2],longitudes[-2])
                                if loc in rest_points:
                                    rest_points[loc]+=diff.seconds
                                else:
                                    rest_points[loc] = diff.seconds
                                    
                                    
                                loc = (round(latitudes[-2],3),round(longitudes[-2],3))
                                if loc in rest_points2:
                                    rest_points2[loc]+=1
                                else:
                                    rest_points2[loc] = 1


                                for hour in hours:
                                    if hour==hr:
                                        if loc in rest_points_time[hr]:
                                            rest_points_time[hr][loc] += 1
                                        else:
                                            rest_points_time[hr][loc] = 1
                                            
                                check = True
                                
                                if check_initial is False:
                                    initial_rest[hr-start_hr]+=1
                                    check_initial = True
        
        if(check):
            days+=1
                                            
                          

        if(count==400):
            print(days)
            break


# Input for Rider number

In [40]:
print("Enter Rider Number:")
num = input()
file_reading2(num)

Enter Rider Number:
4


In [41]:
print(rest_patterns)

{'600': 56, '900': 84, '300': 122, '1800': 62}


# Plotting Pie Graph for rest durations

In [42]:
import plotly.graph_objects as go
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

x=[]
y=[]
for key in rest_patterns:
    x.append(key)
    y.append(rest_patterns[key])

fig = go.Figure(data=[go.Pie(labels=x,
                             values=y,
                             insidetextorientation='radial')])
fig.update_traces(hoverinfo='label+value', textinfo='label+percent', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

# Input for Rider number

In [43]:
print("Enter Rider Number:")
num = input()
file_reading(num)

Enter Rider Number:
4
178


# Distance between 2 geolocation points

In [206]:
def Haversine_distance(lat1,lat2,lon1,lon2):

    r = 6373.0
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1

    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

    c = 2 * math.asin(math.sqrt(a))
    distance = r * c

    return distance*1000

# Sorting dictionary with value

In [44]:
rest_points = sorted(rest_points.items(), key = 
             lambda kv:(kv[1], kv[0]),reverse=True)

# Initializing colors for map plotting

In [46]:
from random import randint
colors = []

for i in range(101):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

# Plotting top rest spots

In [47]:
latitude = []
longitude = []
pause = []
fig = plt.figure(figsize=(10, 10))
count=0
marker = "v"
for loc in rest_points[:min(50,len(rest_points))]:
    
    latitude.append(loc[0][0])
    longitude.append(loc[0][1])
    pause.append(loc[1])

    plt.scatter(loc[0][0],loc[0][1],s=300,marker=marker,color=colors[count],alpha=0.5, label=loc[1])
    count+=1


mplleaflet.display(fig=fig)

In [48]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
        lat=latitude,
        lon=longitude,
        mode='markers',
       marker=go.scattermapbox.Marker(
            size=15,
           color='yellow'
        ),
        text=pause,
        hoverinfo='text'
    ))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "stamen-terrain",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1},
    height = 1000)

fig.show()

# Processing dictionaries for plotting

In [49]:
final_time = {}
for hour in hours:
    maxi = sorted(rest_points_time[hour].items(), key = 
         lambda kv:(kv[1], kv[0]),reverse=True)
    if(len(maxi)>0):
        final_time[hour] = str(maxi[0][0]) + " " + str(maxi[0][1])
    

In [50]:
rest_points2 = sorted(rest_points2.items(), key = 
             lambda kv:(kv[1], kv[0]),reverse=True)

# Plotting bar graph for "location of a rest spot" vs "frequency of the spot"

In [51]:
import plotly.graph_objects as go

x=[]
y=[]
for tup in rest_points2[:min(10,len(rest_points2))]:
    x.append(str(tup[0][0])+","+str(tup[0][1]))
    y.append(tup[1])
    
print(x)
print(y)
    
fig = go.Figure([go.Bar(x=x,y=y)])


fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(xaxis_title="location",yaxis_title="count")
fig.show()

['46.564,15.625', '46.558,15.648', '46.47,15.657', '46.533,15.599', '46.558,15.647', '46.469,15.657', '46.533,15.6', '46.499,15.649', '46.591,15.03', '46.565,15.635']
[28, 20, 18, 12, 10, 10, 8, 8, 6, 6]


In [52]:
print(final_time)

{5: '(47.451, 12.337) 1', 6: '(47.64, 13.618) 1', 7: '(45.884, 10.846) 2', 8: '(46.784, 15.511) 1', 9: '(46.533, 15.6) 3', 10: '(46.47, 15.657) 4', 11: '(46.591, 15.03) 3', 12: '(46.564, 15.625) 5', 13: '(46.47, 15.657) 5', 14: '(46.564, 15.625) 4', 15: '(46.564, 15.625) 3', 16: '(46.564, 15.625) 5', 17: '(46.564, 15.625) 5', 18: '(46.558, 15.648) 7', 19: '(46.558, 15.647) 2', 20: '(46.559, 15.645) 1', 21: '(46.558, 15.648) 2'}


# Plotting bar graph for "hourly interval" vs " count of rest spot occurence in the interval"

In [53]:
import plotly.graph_objects as go
x=[]
y=[]
z=[]
for key in final_time:
    x.append(key)
    z.append(final_time[key].split(")")[0])
    y.append(final_time[key].split(")")[-1])


fig = go.Figure([go.Bar(x=x,y=y,text=z)])


fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(xaxis_title="hour",yaxis_title="count")
fig.show()

# Plotting bar graph for "hour of the exercise" vs "count of first rest taken in that hour"

In [54]:
import plotly.graph_objects as go
x=[]
y=[]
z=[]
for key in initial_rest:
    x.append(key)
    y.append(initial_rest[key])


fig = go.Figure([go.Bar(x=x,y=y)])


fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(xaxis_title="hour",yaxis_title="count")
fig.show()

# Average speed

In [204]:
def file_reading3(num):

    directory = "./"+"Sport"+"/"+"Rider"+str(num)

    count = 0
    total = 0

    for hour in hours:
        average_speed[hour] = 0

    for path in pathlib.Path(directory).iterdir():
        count+=1
        with open(path, 'r') as gpx_file:
            gpx = parser.parse(gpx_file)

            latitudes = []
            longitudes = []
            dates = []

            for track in gpx:
                for segment in track:
                    for point in segment:
                        total+=1
                        latitudes.append(point.latitude)
                        longitudes.append(point.longitude)
                        dates.append(point.time)


                        if(len(latitudes)>1):

                            l1 = str(dates[-2]).split(" ")
                            l2 = str(dates[-1]).split(" ")

                            d1 = l1[0]
                            d2 = l2[0]
                            t1 = l1[1]
                            t2 = l2[1]

                            date1 = dt(int(d1.split("-")[0]),int(d1.split("-")[1]),int(d1.split("-")[2]),
                                       int(t1.split(":")[0]),int(t1.split(":")[1]),int(t1.split(":")[2]))

                            date2 = dt(int(d2.split("-")[0]),int(d2.split("-")[1]),int(d2.split("-")[2]),
                                       int(t2.split(":")[0]),int(t2.split(":")[1]),int(t2.split(":")[2]))

                            diff = date2 - date1

                            hr1 = int(t1.split(":")[0])
                            hr2 = int(t2.split(":")[1])
                            
                            for hour in hours:
                                if hr1==hour and hr2==hour:
                                    average_speed[hour]+=Haversine_distance(latitudes[-2],latitudes[-1],
                                                                           longitudes[-2],longitudes[-1])
                                    
                                                     

        if(count==400):
            print(days)
            break


In [207]:
print("Enter the number of rider:")
num = input()
file_reading3(num)

Enter the number of rider:
4
0


In [212]:
import plotly.graph_objects as go

x=[]
y=[]
for key in average_speed:
    x.append(key)
    y.append(average_speed[key]/60)
    
fig = go.Figure([go.Bar(x=x,y=y)])


fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(xaxis_title="hour",yaxis_title="speed",title="Average speed of rider in m/sec")
fig.show()